In [3]:
meaningless_list = [
    'были', 'если', 'было',
    'этот', 'тебе', 'очень',
    'чего', 'себе', 'есть', 'сама',
    'столько', 'когда', 'каким',
    'даже', 'хотя', 'свое', 'своё',
]

meaningless = set(meaningless_list)

In [4]:
from FIN.fin import FIN
from PrefixSpan.prefixspan import PrefixSpan

In [5]:
import pandas as pd
file = 'repetitors_1.xlsx'
table_raw = pd.concat(pd.read_excel(file, sheet_name=None), ignore_index=True)

table_textless = table_raw.drop(['text' ,'Column1', 'author', 'date', 'id', 'reply', 'tutorlink', 'tutorname'], axis=1)
table = table_raw.drop(['Column1', 'author', 'date', 'id', 'reply', 'tutorlink', 'tutorname'], axis=1)

In [6]:
DB_textless = [[row.discipline, row.mark] for index, row in table_textless.iterrows()]

In [7]:
# Ищем правила только среди предмента и оценки

freqs = FIN(DB_textless, 0.01)
freqs_textless = freqs
 
for pat in freqs:
    print(pat.itemset)

['вполне хорошо']
['почти отлично']
['хорошо']
['немецкий язык']
['начальная школа']
['не указана']
['логопеды']
['география']
['химия']
['психология']
['испанский язык']
['информатика']
['биология']
['физика']
['обществознание']
['английский язык']
['русский язык']
['отлично']
['лучше не бывает!']
['математика']
['ужасно']
['лучше не бывает!', 'английский язык']
['лучше не бывает!', 'русский язык']
['математика', 'лучше не бывает!']
['математика', 'отлично']
['отлично', 'английский язык']
['отлично', 'русский язык']
['ужасно', 'русский язык']
['ужасно', 'математика']
['ужасно', 'обществознание']
['ужасно', 'психология']
['ужасно', 'физика']
['ужасно', 'английский язык']
['ужасно', 'биология']
['ужасно', 'информатика']
['ужасно', 'испанский язык']
['ужасно', 'география']


In [23]:
from string import punctuation
from re import split

split_pat = '[' + punctuation + ' ]'


DB_FIN = []
DB_prefixspan = []

for index, row in table[::15].iterrows():
    current_row = [row.discipline, row.mark]
    
    words = [word.lower()
             for word in split(split_pat, str(row.text))
             if (len(word) > 3 and word.lower() not in meaningless) or word.lower() in ['огэ', 'егэ']]
    
    DB_FIN.append(current_row + words)

    if words:
        DB_prefixspan.append([ words ])

In [39]:
# Добавляем текст

freqs = FIN(DB_FIN, 0.17)
freqs_text = freqs

for pat in freqs:
    print(pat.itemset)
    

['обществознание']
['репетитор']
['математика']
['лучше не бывает!']
['спасибо']
['просто']
['человек']
['преподаватель']
['огэ']
['материал']
['отлично']
['довольны']
['ребенку']
['большой']
['занятия']
['ужасно']
['занятия', 'ребенку']
['занятия', 'большой']
['большой', 'ребенку']
['ужасно', 'ребенку']
['ужасно', 'просто']
['ужасно', 'занятия']
['ужасно', 'огэ']
['ужасно', 'большой']
['ужасно', 'занятия', 'ребенку']
['ужасно', 'занятия', 'большой']
['занятия', 'большой', 'ребенку']
['ужасно', 'большой', 'ребенку']
['ужасно', 'занятия', 'большой', 'ребенку']


In [ ]:
# Обработка текста примечания с помощью prefixspan, который ищет sequential patterns
# Тут не очень интересный датасет для данного алгоритма поэтому и результаты получаются не особо интересные


freqs = PrefixSpan(DB_prefixspan, 0.1)


for pat in freqs:
    print(pat.sequence)

In [21]:
# Ассоциативные правила по первому запуску

array = []

for i, pat1 in enumerate(freqs_textless):
    for j, pat2 in enumerate(freqs_textless):
        if i == j:
            continue
            
    
        itemset1 = pat1.itemset
        itemset2 = pat2.itemset
        
        if set(itemset1) & set(itemset2) != set():
            continue
            
        for pat in freqs_textless:
            if set(pat.itemset) == set(itemset1) | set(itemset2):
                
                array.append({
                    'rule' : str(itemset1) + ' -> ' + str(itemset2),
                    'confidence' : pat.support / pat1.support
                })
                
            
                #print(itemset1, ' -> ', itemset2, end='\t\t')
                #print('confidence:', pat.support / pat1.support)
                
df = pd.DataFrame(array)
df

,rule,confidence
0,['география'] -> ['ужасно'],0.883949
1,['психология'] -> ['ужасно'],0.894461
2,['испанский язык'] -> ['ужасно'],0.835128
3,['информатика'] -> ['ужасно'],0.770069
4,['биология'] -> ['ужасно'],0.813420
5,['физика'] -> ['ужасно'],0.787148
6,['обществознание'] -> ['ужасно'],0.853677
7,['английский язык'] -> ['отлично'],0.377398
8,['английский язык'] -> ['лучше не бывает!'],0.366156
9,['английский язык'] -> ['ужасно'],0.157908


In [37]:
# Ассоциативные правила по второму запуску

array = []

for i, pat1 in enumerate(freqs_text):
    for j, pat2 in enumerate(freqs_text):
        if i == j:
            continue
            
    
        itemset1 = pat1.itemset
        itemset2 = pat2.itemset
        
        if set(itemset1) & set(itemset2) != set():
            continue
            
        for pat in freqs_text:
            if set(pat.itemset) == set(itemset1) | set(itemset2):
                
                array.append({
                    'rule' : str(itemset1) + ' -> ' + str(itemset2),
                    'confidence' : pat.support / pat1.support
                })
                
                #print(pat.itemset, pat.support, pat1.itemset, pat1.support, pat2.itemset, pat2.support)
                #print(itemset1, ' -> ', itemset2, end='\t\t')
                #print('confidence:', pat.support / pat1.support)
                
pd.set_option('display.max_rows', None)
df = pd.DataFrame(array)
df

,rule,confidence
0,['просто'] -> ['ужасно'],0.831854
1,['огэ'] -> ['ужасно'],0.868219
2,['ребенку'] -> ['большой'],0.837566
3,['ребенку'] -> ['занятия'],0.890414
4,['ребенку'] -> ['ужасно'],0.834592
5,"['ребенку'] -> ['занятия', 'большой']",0.832533
6,"['ребенку'] -> ['ужасно', 'занятия']",0.832761
7,"['ребенку'] -> ['ужасно', 'большой']",0.832533
8,"['ребенку'] -> ['ужасно', 'занятия', 'большой']",0.832533
9,['большой'] -> ['ребенку'],0.763663
